In [2]:
import pandas as pd
from collections import Counter
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import *

In [3]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [4]:
import networkx as nx
import dgl

In [5]:
import networkx
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import *
from torch_geometric.loader import NeighborSampler, NeighborLoader
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, ResGatedGraphConv, GATv2Conv, SAGEConv, GENConv, DeepGCNLayer, PairNorm, GINConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import torch.nn.functional as F
from imblearn.under_sampling import RandomUnderSampler
pd.options.mode.use_inf_as_na = True
from collections import Counter
from sklearn.feature_selection import SelectFromModel
import torch.nn as nn
import time
import pickle
from torch.nn import LayerNorm, Linear, ReLU
from torch_scatter import scatter
from tqdm import tqdm
from torch_geometric.loader import RandomNodeSampler
import math
import copy

In [6]:
np.random.seed(2022)
torch.manual_seed(2022)

In [75]:
df = pd.read_pickle('cicids2017_all_data_not_web_attack_s_benign.pkl')

In [76]:
df

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443.0,192.168.10.5,54865.0,6.0,7/7/2017 3:30,3.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80.0,192.168.10.5,55054.0,6.0,7/7/2017 3:30,109.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80.0,192.168.10.5,55055.0,6.0,7/7/2017 3:30,52.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443.0,192.168.10.16,46236.0,6.0,7/7/2017 3:30,34.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443.0,192.168.10.5,54863.0,6.0,7/7/2017 3:30,3.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92885,172.16.0.1-192.168.10.50-36214-80-6,172.16.0.1,36214.0,192.168.10.50,80.0,6.0,6/7/2017 10:42,5005388.0,5.0,3.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Web Attack  Sql Injection
92984,172.16.0.1-192.168.10.50-36242-80-6,172.16.0.1,36242.0,192.168.10.50,80.0,6.0,6/7/2017 10:42,5016759.0,5.0,5.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Web Attack  Sql Injection
93233,172.16.0.1-192.168.10.50-36244-80-6,172.16.0.1,36244.0,192.168.10.50,80.0,6.0,6/7/2017 10:42,5009163.0,4.0,4.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Web Attack  Sql Injection
93242,172.16.0.1-192.168.10.50-36242-80-6,172.16.0.1,36242.0,192.168.10.50,80.0,6.0,6/7/2017 10:42,73.0,1.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Web Attack  Sql Injection


In [77]:
Counter(df[' Label'] != 'BENIGN')

Counter({False: 2104911, True: 557646})

In [78]:
Counter(df[' Label'])

Counter({'BENIGN': 2104911,
         'DDoS': 128027,
         'PortScan': 158930,
         'Bot': 1966,
         'Infiltration': 36,
         'FTP-Patator': 7938,
         'SSH-Patator': 5897,
         'DoS slowloris': 5796,
         'DoS Slowhttptest': 5499,
         'DoS Hulk': 231073,
         'DoS GoldenEye': 10293,
         'Heartbleed': 11,
         'Web Attack \x96 Brute Force': 1507,
         'Web Attack \x96 XSS': 652,
         'Web Attack \x96 Sql Injection': 21})

In [79]:
df = df.dropna()

In [80]:
LABELS_7 = {
    'BENIGN': 0,
    
     'DDoS': 1,
     'DoS slowloris': 1,
     'DoS Slowhttptest': 1,
     'DoS Hulk': 1,
     'DoS GoldenEye': 1,
     'Heartbleed': 1,
    
     'PortScan': 2,
    
     'Bot': 3,
    
     'Infiltration': 4,
    
     'FTP-Patator': 5,
     'SSH-Patator': 5,
    
    'Web Attack \x96 Brute Force':6,
    'Web Attack \x96 XSS': 6,
    'Web Attack \x96 Sql Injection': 6
}

In [81]:
from imblearn.under_sampling import RandomUnderSampler

In [82]:
len(df.columns)

85

In [83]:
x = df.drop(columns=[' Label'])
y = df[' Label']

In [84]:
len(y[y!='BENIGN'])

556556

In [85]:
Counter(y)

Counter({'BENIGN': 2103269,
         'DDoS': 128025,
         'PortScan': 158804,
         'Bot': 1956,
         'Infiltration': 36,
         'FTP-Patator': 7935,
         'SSH-Patator': 5897,
         'DoS slowloris': 5796,
         'DoS Slowhttptest': 5499,
         'DoS Hulk': 230124,
         'DoS GoldenEye': 10293,
         'Heartbleed': 11,
         'Web Attack \x96 Brute Force': 1507,
         'Web Attack \x96 XSS': 652,
         'Web Attack \x96 Sql Injection': 21})

In [86]:
rus = RandomUnderSampler(random_state=1, sampling_strategy={
        'BENIGN': 50000
})
LABELS_15 = {
    'BENIGN': 0,
     'DDoS': 1,
     'PortScan': 2,
     'Bot': 3,
     'Infiltration': 4,
     'FTP-Patator': 5,
     'SSH-Patator': 6,
     'DoS slowloris': 7,
     'DoS Slowhttptest': 8,
     'DoS Hulk': 9,
     'DoS GoldenEye': 10,
     'Heartbleed': 11,
    'Web Attack \x96 Brute Force':12,
    'Web Attack \x96 XSS': 13,
    'Web Attack \x96 Sql Injection': 14
}

In [87]:
df

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443.0,192.168.10.5,54865.0,6.0,7/7/2017 3:30,3.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80.0,192.168.10.5,55054.0,6.0,7/7/2017 3:30,109.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80.0,192.168.10.5,55055.0,6.0,7/7/2017 3:30,52.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443.0,192.168.10.16,46236.0,6.0,7/7/2017 3:30,34.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443.0,192.168.10.5,54863.0,6.0,7/7/2017 3:30,3.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92885,172.16.0.1-192.168.10.50-36214-80-6,172.16.0.1,36214.0,192.168.10.50,80.0,6.0,6/7/2017 10:42,5005388.0,5.0,3.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Web Attack  Sql Injection
92984,172.16.0.1-192.168.10.50-36242-80-6,172.16.0.1,36242.0,192.168.10.50,80.0,6.0,6/7/2017 10:42,5016759.0,5.0,5.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Web Attack  Sql Injection
93233,172.16.0.1-192.168.10.50-36244-80-6,172.16.0.1,36244.0,192.168.10.50,80.0,6.0,6/7/2017 10:42,5009163.0,4.0,4.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Web Attack  Sql Injection
93242,172.16.0.1-192.168.10.50-36242-80-6,172.16.0.1,36242.0,192.168.10.50,80.0,6.0,6/7/2017 10:42,73.0,1.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Web Attack  Sql Injection


In [88]:
x, y = rus.fit_resample(x, y)

In [89]:
x.shape

(606556, 84)

In [90]:
Counter(y)

Counter({'BENIGN': 50000,
         'Bot': 1956,
         'DDoS': 128025,
         'DoS GoldenEye': 10293,
         'DoS Hulk': 230124,
         'DoS Slowhttptest': 5499,
         'DoS slowloris': 5796,
         'FTP-Patator': 7935,
         'Heartbleed': 11,
         'Infiltration': 36,
         'PortScan': 158804,
         'SSH-Patator': 5897,
         'Web Attack \x96 Brute Force': 1507,
         'Web Attack \x96 Sql Injection': 21,
         'Web Attack \x96 XSS': 652})

In [91]:
label = [LABELS_15[v] for v in y]

In [92]:
Counter(label), len(label)

(Counter({0: 50000,
          3: 1956,
          1: 128025,
          10: 10293,
          9: 230124,
          8: 5499,
          7: 5796,
          5: 7935,
          11: 11,
          4: 36,
          2: 158804,
          6: 5897,
          12: 1507,
          14: 21,
          13: 652}),
 606556)

In [93]:
df = x
df[' Label'] = label

In [94]:
df = df.dropna()

In [95]:
df

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.15-217.68.217.19-54607-443-6,217.68.217.19,443.0,192.168.10.15,54607.0,6.0,03/07/2017 04:35:47,332.0,3.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,192.168.10.1-192.168.10.3-53-60233-17,192.168.10.3,60233.0,192.168.10.1,53.0,17.0,7/7/2017 10:18,96026.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,172.217.12.142-192.168.10.15-80-50621-6,172.217.12.142,80.0,192.168.10.15,50621.0,6.0,5/7/2017 9:54,35.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,172.16.0.1-192.168.10.50-31633-80-6,192.168.10.50,80.0,172.16.0.1,31633.0,6.0,7/7/2017 4:09,74154.0,1.0,5.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,192.168.10.9-119.167.164.11-21068-443-6,119.167.164.11,443.0,192.168.10.9,21068.0,6.0,5/7/2017 2:54,513898.0,4.0,5.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606551,172.16.0.1-192.168.10.50-36182-80-6,172.16.0.1,36182.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,5029253.0,9.0,7.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606552,172.16.0.1-192.168.10.50-36184-80-6,172.16.0.1,36184.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,5013370.0,7.0,6.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606553,172.16.0.1-192.168.10.50-36182-80-6,172.16.0.1,36182.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,18.0,1.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606554,172.16.0.1-192.168.10.50-36184-80-6,172.16.0.1,36184.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,13.0,1.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13


In [96]:
df.to_pickle('cic2017_normal_5w.pkl')

In [7]:
df = pd.read_pickle('cic2017_normal_5w.pkl')

In [8]:
df

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.15-217.68.217.19-54607-443-6,217.68.217.19,443.0,192.168.10.15,54607.0,6.0,03/07/2017 04:35:47,332.0,3.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,192.168.10.1-192.168.10.3-53-60233-17,192.168.10.3,60233.0,192.168.10.1,53.0,17.0,7/7/2017 10:18,96026.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,172.217.12.142-192.168.10.15-80-50621-6,172.217.12.142,80.0,192.168.10.15,50621.0,6.0,5/7/2017 9:54,35.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,172.16.0.1-192.168.10.50-31633-80-6,192.168.10.50,80.0,172.16.0.1,31633.0,6.0,7/7/2017 4:09,74154.0,1.0,5.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,192.168.10.9-119.167.164.11-21068-443-6,119.167.164.11,443.0,192.168.10.9,21068.0,6.0,5/7/2017 2:54,513898.0,4.0,5.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606551,172.16.0.1-192.168.10.50-36182-80-6,172.16.0.1,36182.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,5029253.0,9.0,7.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606552,172.16.0.1-192.168.10.50-36184-80-6,172.16.0.1,36184.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,5013370.0,7.0,6.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606553,172.16.0.1-192.168.10.50-36182-80-6,172.16.0.1,36182.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,18.0,1.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606554,172.16.0.1-192.168.10.50-36184-80-6,172.16.0.1,36184.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,13.0,1.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13


In [9]:
Counter(label)

NameError: name 'label' is not defined

In [10]:
import networkx as nx
def get_lg_graph(src_ip, src_port, dst_ip, dst_port, features, label):
    nodes_to_no_dict = {}
    src_ip = np.array(src_ip)
    dst_ip = np.array(dst_ip)
    src_port = np.array(src_port)
    dst_port = np.array(dst_port)
    src_nodes = list(zip(src_ip, src_port))
    dst_nodes = list(zip(dst_ip, dst_port))
    socket_dict = list(zip(src_nodes, dst_nodes))
    all_nodes = set(src_nodes).union(set(dst_nodes))
    for index,value in enumerate(all_nodes):
        nodes_to_no_dict[value] = index
    src_index = [nodes_to_no_dict[s] for s in src_nodes]
    dst_index = [nodes_to_no_dict[s] for s in dst_nodes]
    index = [src_index, dst_index]
    index = np.array(index)
    g = nx.MultiGraph()
    for i in range(len(src_index)):
        g.add_edge(src_index[i], dst_index[i], e_id=i)
    print(len(g.edges))
    features_map = dict()
    it =  iter(g.edges.data())
    for edge in g.edges:
        edge = tuple((edge[0], edge[1], edge[2]))
        edge_invere = tuple((edge[1], edge[0], edge[2]))
        data = next(it)
        features_map[edge] = data[2]['e_id']
        features_map[edge_invere] = data[2]['e_id']
    print('开始转换')
    lg = nx.line_graph(g)
    print('转换结束')
    src_edge = []
    dst_edge = []
    for edge in lg.edges():
        src, dst = edge[0], edge[1]
        src_idx = features_map[src]
        dst_idx = features_map[dst]
        src_edge.append(src_idx)
        dst_edge.append(dst_idx)
    index = [src_edge, dst_edge]
    index = torch.LongTensor(index)
    features = features.astype(np.float)
    features = StandardScaler().fit_transform(features)
    features = torch.FloatTensor(features)
    num_nodes = len(label)
    label = label.astype(np.int)
    label = torch.LongTensor(label)
    data = Data(x=features, edge_index=index, y=label, num_nodes=num_nodes)
    return data

In [11]:
df

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.15-217.68.217.19-54607-443-6,217.68.217.19,443.0,192.168.10.15,54607.0,6.0,03/07/2017 04:35:47,332.0,3.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,192.168.10.1-192.168.10.3-53-60233-17,192.168.10.3,60233.0,192.168.10.1,53.0,17.0,7/7/2017 10:18,96026.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,172.217.12.142-192.168.10.15-80-50621-6,172.217.12.142,80.0,192.168.10.15,50621.0,6.0,5/7/2017 9:54,35.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,172.16.0.1-192.168.10.50-31633-80-6,192.168.10.50,80.0,172.16.0.1,31633.0,6.0,7/7/2017 4:09,74154.0,1.0,5.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,192.168.10.9-119.167.164.11-21068-443-6,119.167.164.11,443.0,192.168.10.9,21068.0,6.0,5/7/2017 2:54,513898.0,4.0,5.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606551,172.16.0.1-192.168.10.50-36182-80-6,172.16.0.1,36182.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,5029253.0,9.0,7.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606552,172.16.0.1-192.168.10.50-36184-80-6,172.16.0.1,36184.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,5013370.0,7.0,6.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606553,172.16.0.1-192.168.10.50-36182-80-6,172.16.0.1,36182.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,18.0,1.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606554,172.16.0.1-192.168.10.50-36184-80-6,172.16.0.1,36184.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,13.0,1.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13


In [12]:
data = copy.copy(df)

In [13]:
src_ip = np.array(data[' Source IP'])
dst_ip = np.array(data[' Destination IP'])
src_port = np.array(data[' Source Port'])
dst_port = np.array(data[' Destination Port'])
src_nodes = list(zip(src_ip, src_port))
dst_nodes = list(zip(dst_ip, dst_port))

In [14]:
nodes_to_no_dict = {}
socket_dict = list(zip(src_nodes, dst_nodes))
all_nodes = set(src_nodes).union(set(dst_nodes))
for index,value in enumerate(all_nodes):
    nodes_to_no_dict[value] = index
src_index = [nodes_to_no_dict[s] for s in src_nodes]
dst_index = [nodes_to_no_dict[s] for s in dst_nodes]
data['saddr'] = src_index
data['daddr'] = dst_index

In [105]:
src_nodes

[('217.68.217.19', 443.0),
 ('192.168.10.3', 60233.0),
 ('172.217.12.142', 80.0),
 ('192.168.10.50', 80.0),
 ('119.167.164.11', 443.0),
 ('192.168.10.9', 3614.0),
 ('192.168.10.3', 61922.0),
 ('192.168.10.15', 52325.0),
 ('192.168.10.3', 62151.0),
 ('192.168.10.15', 55895.0),
 ('192.168.10.15', 60160.0),
 ('192.168.10.3', 60089.0),
 ('192.168.10.8', 54786.0),
 ('192.168.10.3', 61326.0),
 ('192.168.10.5', 57509.0),
 ('192.168.10.5', 52500.0),
 ('192.168.10.25', 61989.0),
 ('192.168.10.14', 51703.0),
 ('192.168.10.15', 54372.0),
 ('192.168.10.14', 63796.0),
 ('192.168.10.3', 61215.0),
 ('120.55.226.124', 443.0),
 ('192.168.10.17', 56519.0),
 ('192.168.10.9', 1043.0),
 ('192.168.10.9', 61127.0),
 ('192.168.10.12', 47664.0),
 ('192.168.10.17', 13071.0),
 ('192.168.10.50', 80.0),
 ('192.168.10.15', 52190.0),
 ('192.168.10.5', 61138.0),
 ('192.168.10.8', 50746.0),
 ('192.168.10.3', 61566.0),
 ('192.168.10.14', 64118.0),
 ('63.144.176.10', 80.0),
 ('192.168.10.14', 58170.0),
 ('192.168.10.8',

In [11]:
np.save('cic2017_nodes_to_no_dict',nodes_to_no_dict)

In [12]:
node_index = [nodes_to_no_dict[s] for s in all_nodes]

In [13]:
num_embeddings = len(all_nodes)

In [14]:
num_embeddings

76668

In [67]:
embedding_dim = 78

In [68]:
embedector = torch.nn.Embedding(num_embeddings, embedding_dim)

In [71]:
node_index = torch.LongTensor(node_index)

In [85]:
src_node_index = torch.LongTensor(src_index)

In [87]:
src_node_features = embedector(src_node_index)

In [88]:
src_node_features.shape

torch.Size([2659825, 78])

In [73]:
node_features.shape

torch.Size([435887, 78])

In [74]:
node_features

tensor([[-0.9788, -1.5154, -0.8222,  ...,  1.4420,  0.5218, -0.1056],
        [ 0.0886,  0.9826, -1.8965,  ...,  0.9200, -0.8281, -0.1903],
        [ 0.8735, -0.4488,  0.0056,  ...,  1.7360, -1.0492,  0.0534],
        ...,
        [ 2.2031, -1.3432,  1.6674,  ...,  0.5196,  0.9907, -0.6244],
        [ 1.6498, -0.1467, -0.2761,  ...,  0.1595,  2.4443,  0.0598],
        [ 1.4951,  0.8378, -0.7953,  ...,  0.0194, -0.3393, -0.4021]],
       grad_fn=<EmbeddingBackward0>)

In [83]:
torch.save(node_features, 'cic2017_node_features')

In [181]:
data[' Label']

0          0
1          0
2          0
3          0
4          0
          ..
149598    13
149599    13
149600    13
149601    13
149602    13
Name:  Label, Length: 149603, dtype: int64

In [182]:
data

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,saddr,daddr
0,192.168.10.15-217.68.217.19-54607-443-6,217.68.217.19,443.0,192.168.10.15,54607.0,6.0,03/07/2017 04:35:47,332.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,9985,28807
1,192.168.10.1-192.168.10.3-53-60233-17,192.168.10.3,60233.0,192.168.10.1,53.0,17.0,7/7/2017 10:18,96026.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,62643,18246
2,172.217.12.142-192.168.10.15-80-50621-6,172.217.12.142,80.0,192.168.10.15,50621.0,6.0,5/7/2017 9:54,35.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,40277,41277
3,172.16.0.1-192.168.10.50-31633-80-6,192.168.10.50,80.0,172.16.0.1,31633.0,6.0,7/7/2017 4:09,74154.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,47675,60959
4,192.168.10.9-119.167.164.11-21068-443-6,119.167.164.11,443.0,192.168.10.9,21068.0,6.0,5/7/2017 2:54,513898.0,4.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,31494,61305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149598,172.16.0.1-192.168.10.50-52320-80-6,172.16.0.1,52320.0,192.168.10.50,80.0,6.0,6/7/2017 10:16,5607142.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,28890,47675
149599,172.16.0.1-192.168.10.50-56130-80-6,172.16.0.1,56130.0,192.168.10.50,80.0,6.0,6/7/2017 10:22,5225282.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,32502,47675
149600,172.16.0.1-192.168.10.50-58224-80-6,172.16.0.1,58224.0,192.168.10.50,80.0,6.0,6/7/2017 10:25,5679516.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,49552,47675
149601,172.16.0.1-192.168.10.50-35560-80-6,172.16.0.1,35560.0,192.168.10.50,80.0,6.0,6/7/2017 10:34,5230205.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,70749,47675


In [106]:
data.drop(columns=[' Source IP',' Source Port',' Destination IP', ' Destination Port'],inplace=True)
label_ground_truth = data[["saddr", "daddr"," Label"]]

In [107]:
data.drop(columns=['Flow ID', ' Timestamp'],inplace=True)

In [108]:
data = data.reset_index()
data.replace([np.inf, -np.inf], np.nan,inplace = True)

In [109]:
data.fillna(0,inplace = True)

In [110]:
label_ground_truth = data[["saddr", "daddr"," Label"]]

In [111]:
data.drop(columns=['index'],inplace=True)

In [112]:
scaler = StandardScaler()
cols_to_norm = list(set(list(data.iloc[:, :-3].columns ))  - set(list([' Label'])) )
data[cols_to_norm] = scaler.fit_transform(data[cols_to_norm])

In [113]:
label_ground_truth = label_ground_truth.rename(columns={' Label':'label'})

In [114]:
import socket
import struct
import random

In [115]:
def get_g(src_index, dst_index):
    index = [src_index, dst_index]
    index = np.array(index).flatten()
    index = list(index)
    print(index)
    g = nx.MultiGraph()
    for v in index:
        g.add_node(v, node_id=v)
    for i in range(len(src_index)):
        g.add_edge(src_index[i], dst_index[i], e_id=i, src_index=src_index[i], dst_index=dst_index[i])
    return g

In [116]:
g = get_g(src_index, dst_index)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [117]:
g = g.to_directed()

In [118]:
g = dgl.from_networkx(g, edge_attrs=['e_id','src_index','dst_index'], node_attrs=['node_id'])

In [119]:
g

Graph(num_nodes=99030, num_edges=1213112,
      ndata_schemes={'node_id': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'e_id': Scheme(shape=(), dtype=torch.int64), 'src_index': Scheme(shape=(), dtype=torch.int64), 'dst_index': Scheme(shape=(), dtype=torch.int64)})

In [120]:
X_train, X_test, y_train, y_test = train_test_split(
     data, label_ground_truth, test_size=0.3, random_state=42, stratify=label_ground_truth.label)

In [121]:
X_train['h'] = X_train[ cols_to_norm ].values.tolist()

/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [122]:
X_test['h'] = X_test[ cols_to_norm ].values.tolist()

/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [123]:
X_train = X_train.rename(columns={' Label':'label'})

In [124]:
X_test = X_test.rename(columns={' Label':'label'})

In [125]:
X_train

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label,saddr,daddr,h
126797,-0.192293,-0.548854,-0.012526,-0.124293,-0.031601,-0.095464,-0.382453,0.050009,-0.254940,-0.417814,...,-0.146150,-0.133139,-0.630776,-0.172807,-0.643116,-0.606413,1,6361,25159,"[-0.7438269061427828, -0.12429335133569537, -0..."
123035,-0.192293,1.380809,0.158038,0.049634,-0.026771,0.112537,-0.337366,-0.084566,-0.243549,-0.369075,...,3.152692,4.076860,1.403655,-0.172807,1.346287,1.430919,1,28212,25159,"[1.4787560409954443, 0.049633553642985724, -0...."
81426,-0.192293,1.187127,0.158038,0.049634,-0.026771,0.112429,-0.337366,-0.084566,-0.243549,-0.369075,...,-0.144944,-0.131600,0.337402,5.066822,1.038901,-0.391730,1,16555,25159,"[0.7304334023126547, 0.049633553642985724, -0...."
374349,-0.192293,1.343827,0.115397,0.049634,0.010659,0.112322,0.577254,-0.084566,0.171428,0.572171,...,-0.146144,-0.133131,1.476582,-0.172807,1.417601,1.503951,9,98140,25159,"[1.4813396999094552, 0.049633553642985724, -0...."
315127,-0.192293,-0.667905,-0.097808,-0.124293,-0.035223,-0.095464,-0.401776,-0.084566,-0.323289,-0.417814,...,-0.146150,-0.133139,-0.630776,-0.172807,-0.643116,-0.606413,9,42773,25159,"[-0.7438269061427828, -0.12429335133569537, -0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261361,-0.192293,1.757875,0.158038,0.084419,0.012621,0.112322,0.599798,-0.084566,0.128100,0.527117,...,-0.144947,-0.131604,1.909119,-0.172807,1.840564,1.937104,9,93628,25159,"[0.7128458499076351, 0.08441893463872195, -0.2..."
423001,-0.192293,-0.667904,-0.140449,-0.089508,0.043260,-0.095356,1.272881,11.578563,5.600300,-0.417814,...,-0.146150,-0.133139,-0.630776,-0.172807,-0.643116,-0.606413,8,75803,25159,"[-0.7438269061427828, -0.08950797033995915, -0..."
402546,-0.192293,1.346793,0.158038,0.119204,0.019262,0.112322,0.760822,-0.084566,0.190753,0.681865,...,-0.144938,-0.131592,1.479097,-0.172807,1.420060,1.506469,9,36033,25159,"[0.430041889517067, 0.11920431563445817, -0.29..."
210350,-0.192293,1.348452,0.115397,0.049634,0.022432,0.112322,0.809130,-0.084566,0.298362,0.803536,...,-0.143689,-0.129998,1.481612,-0.172807,1.422519,1.508987,9,82779,25159,"[1.4813396999094552, 0.049633553642985724, -0...."


In [126]:
G = nx.from_pandas_edgelist(X_train, "saddr", "daddr", ['h','label','saddr','daddr'], create_using = nx.MultiGraph())

In [127]:
G = G.to_directed()

In [128]:
G = dgl.from_networkx(G, edge_attrs=['h','label','saddr','daddr'])

In [129]:
G

Graph(num_nodes=82830, num_edges=849178,
      ndata_schemes={}
      edata_schemes={'h': Scheme(shape=(78,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'saddr': Scheme(shape=(), dtype=torch.int64), 'daddr': Scheme(shape=(), dtype=torch.int64)})

In [130]:
from dgl.data.utils import save_graphs
save_graphs("./cic2017_Gtrain.bin", [G])

In [ ]:
LG_train = dgl.line_graph(G, shared=True)

In [154]:
LG_train

Graph(num_nodes=139444, num_edges=1492813740,
      ndata_schemes={'h': Scheme(shape=(78,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})

In [169]:
from dgl.data.utils import save_graphs
save_graphs("./cic2017_train_data_lg2.bin", [LG])

In [170]:
Counter(LG_test.ndata['label'].numpy())

Counter({2: 9000,
         9: 9000,
         10: 6176,
         7: 3478,
         1: 9000,
         8: 3300,
         12: 904,
         0: 9000,
         13: 392,
         14: 12,
         6: 3538,
         3: 1174,
         5: 4760,
         4: 22,
         11: 6})

In [171]:
from dgl.data.utils import save_graphs
save_graphs("./cic2017_test_data_lg2.bin", [LG_test])

In [208]:
G = G.to_directed()

In [209]:
G = dgl.from_networkx(G, edge_attrs=['h','label'])

In [210]:
G

Graph(num_nodes=88732, num_edges=279444,
      ndata_schemes={}
      edata_schemes={'h': Scheme(shape=(78,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)})

In [172]:
from dgl.data.utils import save_graphs
save_graphs("./cic2017_data_rus2.bin", [G])

In [131]:
G_test = nx.from_pandas_edgelist(X_test, "saddr", "daddr", ['h','label'], create_using= nx.MultiGraph())

In [132]:
G_test = G_test.to_directed()

In [133]:
G_test = dgl.from_networkx(G_test,edge_attrs=['h','label'])

In [134]:
save_graphs("./cic2017_G_test.bin", [G_test])

In [166]:
LG_test = dgl.line_graph(G_test, shared=True)

In [167]:
LG_test

Graph(num_nodes=59762, num_edges=274995716,
      ndata_schemes={'h': Scheme(shape=(78,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})

In [177]:
save_graphs("./cic2017_test_data_rus2.bin", [G_test])

In [15]:
ori_data = np.array(df)

In [16]:
Counter(df[' Label'])

Counter({0: 50000,
         3: 1956,
         1: 128025,
         10: 10293,
         9: 230124,
         8: 5499,
         7: 5796,
         5: 7935,
         11: 11,
         4: 36,
         2: 158804,
         6: 5897,
         12: 1507,
         14: 21,
         13: 652})

In [17]:
ori_data[:, -1]

array([0, 0, 0, ..., 13, 13, 13], dtype=object)

In [18]:
all_index = np.arange(len(df))

In [19]:
all_index.shape

(606556,)

In [20]:
train_mask, test_mask = train_test_split(all_index, stratify=ori_data[:, -1], test_size=0.3, random_state=41)

In [21]:
train_mask.shape, test_mask.shape

((424589,), (181967,))

In [22]:
# x_train, x_test, y_train, y_test = train_test_split(ori_data, ori_data[:, -1], test_size=0.15,stratify=ori_data[:, -1], random_state=41)

In [23]:
x_train, x_test, y_train, y_test = ori_data[train_mask], ori_data[test_mask],ori_data[:, -1][train_mask],ori_data[:, -1][test_mask]

In [24]:
LABELS_15 = {
    'BENIGN': 0,
     'DDoS': 1,
     'PortScan': 2,
     'Bot': 3,
     'Infiltration': 4,
     'FTP-Patator': 5,
     'SSH-Patator': 6,
     'DoS slowloris': 7,
     'DoS Slowhttptest': 8,
     'DoS Hulk': 9,
     'DoS GoldenEye': 10,
     'Heartbleed': 11,
    'Web Attack \x96 Brute Force':12,
    'Web Attack \x96 XSS': 13,
    'Web Attack \x96 Sql Injection': 14
}

In [25]:
Counter(y_train)

Counter({1: 89617,
         2: 111163,
         0: 35000,
         9: 161087,
         7: 4057,
         8: 3849,
         10: 7205,
         3: 1369,
         5: 5555,
         6: 4128,
         12: 1055,
         13: 456,
         14: 15,
         4: 25,
         11: 8})

In [26]:
Counter(y_test)

Counter({2: 47641,
         1: 38408,
         9: 69037,
         0: 15000,
         10: 3088,
         6: 1769,
         8: 1650,
         5: 2380,
         12: 452,
         7: 1739,
         13: 196,
         3: 587,
         4: 11,
         14: 6,
         11: 3})

In [27]:
x_test

array([['172.16.0.1-192.168.10.50-44511-4446-6', '172.16.0.1', 44511.0,
        ..., 0.0, 0.0, 2],
       ['172.16.0.1-192.168.10.50-49167-80-6', '172.16.0.1', 49167.0,
        ..., 5881892.0, 5881892.0, 1],
       ['172.16.0.1-192.168.10.50-26839-80-6', '172.16.0.1', 26839.0,
        ..., 0.0, 0.0, 1],
       ...,
       ['172.16.0.1-192.168.10.50-60420-80-6', '172.16.0.1', 60420.0,
        ..., 0.0, 0.0, 9],
       ['172.16.0.1-192.168.10.50-32302-80-6', '172.16.0.1', 32302.0,
        ..., 10500000.0, 10500000.0, 1],
       ['172.16.0.1-192.168.10.50-48416-5414-6', '172.16.0.1', 48416.0,
        ..., 0.0, 0.0, 2]], dtype=object)

In [28]:
new_x_test = np.delete(x_test, 6, axis=-1)

In [29]:
new_x_train = np.delete(x_train, 6, axis=-1)

In [30]:
test_src_ip = x_test[:, 1]
test_src_port = x_test[:, 2]
test_dst_ip = x_test[:, 3]
test_dst_port = x_test[:, 4]
test_features = new_x_test[:, 5:-1]
test_label = x_test[:, -1]

In [31]:
test_features.shape

(181967, 78)

In [32]:
train_src_ip = x_train[:, 1]
train_src_port = x_train[:, 2]
train_dst_ip = x_train[:, 3]
train_dst_port = x_train[:, 4]
train_features = new_x_train[:, 5:-1]
train_label = x_train[:, -1]

In [33]:
train_features.shape

(424589, 78)

In [52]:
train_g = get_lg_graph(train_src_ip, train_src_port, train_dst_ip, train_dst_port, train_features, train_label)

NameError: name 'get_lg_graph' is not defined

In [53]:
train_g

NameError: name 'train_g' is not defined

In [56]:
test_lg = get_lg_graph(test_src_ip, test_src_port, test_dst_ip, test_dst_port, test_features, test_label)

2
2
35881


DGLError: Expect number of features to match number of edges. Got 35881 and 71762 instead.

In [53]:
test_lg

Graph(num_nodes=35881, num_edges=1882030,
      ndata_schemes={'x': Scheme(shape=(78,), dtype=torch.float32), 'y': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})

In [182]:
torch.save(train_lg, 'cic2017_directed_line_graph_train_data_15label3')

In [183]:
torch.save(test_lg, 'cic2017_directed_line_graph_test_data_15label3')

In [42]:
#LightGBM

In [153]:
from sklearn.tree import DecisionTreeClassifier

In [154]:
from sklearn.ensemble._forest import RandomForestClassifier

In [35]:
train_features

array([[6.0, 975143.0, 3.0, ..., 0.0, 0.0, 0.0],
       [6.0, 50.0, 1.0, ..., 0.0, 0.0, 0.0],
       [6.0, 5054055.0, 4.0, ..., 0.0, 5053377.0, 5053377.0],
       ...,
       [6.0, 43.0, 1.0, ..., 0.0, 0.0, 0.0],
       [6.0, 10856204.0, 9.0, ..., 0.0, 5853648.0, 5853648.0],
       [6.0, 1.0, 2.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [36]:
torch.save(train_features, 'cicids2017_train_x')
torch.save(train_label, 'cicids2017_train_label')
torch.save(test_features, 'cicids2017_test_x')
torch.save(test_label, 'cicids2017_test_label')

In [34]:
Counter(train_label)

Counter({1: 89617,
         2: 111163,
         0: 35000,
         9: 161087,
         7: 4057,
         8: 3849,
         10: 7205,
         3: 1369,
         5: 5555,
         6: 4128,
         12: 1055,
         13: 456,
         14: 15,
         4: 25,
         11: 8})

In [156]:
train_label = np.asarray(train_label, np.int)

In [157]:
test_label = np.asarray(test_label, np.int)

In [158]:
train_label_bi = np.array(train_label)

In [159]:
test_label_bi = np.array(test_label)

In [160]:
train_label_bi[train_label_bi != 0] = 1

In [161]:
test_label_bi[test_label_bi != 0] = 1

In [162]:
LABELS_7 = {
    0: 0, 
    
    1:1,
    7:1,
    8:1,
    9:1,
    10:1,
    11:1,
    
    2:2,
    3:3,
    4:4, 
    
    5:5,
    6:5,
    
    12:6,
    13:6,
    14:6
}

In [163]:
train_label7 = [LABELS_7[v] for v in train_label]
test_label7 = [LABELS_7[v] for v in test_label]

In [164]:
train_features = np.asarray(train_features, np.float)

In [165]:
test_features = np.asarray(test_features, np.float)

In [166]:
train_features.shape

(424589, 78)

In [68]:
ori_data2 = pd.DataFrame(ori_data)

In [69]:
Counter(ori_data2[84])

Counter({0: 50000,
         3: 1956,
         1: 128025,
         10: 10293,
         9: 230124,
         8: 5499,
         7: 5796,
         5: 7935,
         11: 11,
         4: 36,
         2: 158804,
         6: 5897,
         12: 1507,
         14: 21,
         13: 652})

In [70]:
features_lgbm = ori_data2.drop(columns=[0,1,2,3,4,6,84])

In [71]:
features_lgbm

,5,7,8,9,10,11,12,13,14,15,...,74,75,76,77,78,79,80,81,82,83
0,6,332,3,0,18,0,6,6,6,0,...,2,20,0,0,0,0,0,0,0,0
1,17,96026,1,1,50,252,50,50,50,0,...,0,20,0,0,0,0,0,0,0,0
2,6,35,1,1,6,6,6,6,6,0,...,0,20,0,0,0,0,0,0,0,0
3,6,74154,1,5,6,30,6,6,6,0,...,0,20,0,0,0,0,0,0,0,0
4,6,513898,4,5,24,18,6,6,6,0,...,3,20,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606551,6,5.02925e+06,9,7,1675,5647,559,0,186.111,228.375,...,4,32,0,0,0,0,0,0,0,0
606552,6,5.01337e+06,7,6,740,7037,382,0,105.714,180.674,...,2,32,0,0,0,0,0,0,0,0
606553,6,18,1,1,0,0,0,0,0,0,...,0,32,0,0,0,0,0,0,0,0
606554,6,13,1,1,0,0,0,0,0,0,...,0,32,0,0,0,0,0,0,0,0


In [72]:
labels_lgbm = ori_data2[84]

In [73]:
labels_lgbm

0          0
1          0
2          0
3          0
4          0
          ..
606551    13
606552    13
606553    13
606554    13
606555    13
Name: 84, Length: 606556, dtype: object

In [74]:
Counter(labels_lgbm)

Counter({0: 50000,
         3: 1956,
         1: 128025,
         10: 10293,
         9: 230124,
         8: 5499,
         7: 5796,
         5: 7935,
         11: 11,
         4: 36,
         2: 158804,
         6: 5897,
         12: 1507,
         14: 21,
         13: 652})

In [75]:
x_train_lgm, x_test_lgm, y_train_lgm, y_test_lgm =features_lgbm.iloc[train_mask, :], features_lgbm.iloc[test_mask, :],labels_lgbm[train_mask],labels_lgbm[test_mask]

In [76]:
x_train_lgm

,5,7,8,9,10,11,12,13,14,15,...,74,75,76,77,78,79,80,81,82,83
52651,6,975143,3,6,26,11607,20,0,8.66667,10.2632,...,2,20,0,0,0,0,0,0,0,0
529942,6,50,1,1,0,6,0,0,0,0,...,0,40,0,0,0,0,0,0,0,0
78781,6,5.05406e+06,4,0,24,0,6,6,6,0,...,3,20,678,0,678,678,5.05338e+06,0,5.05338e+06,5.05338e+06
39822,17,61129,4,2,120,264,30,30,30,0,...,3,20,0,0,0,0,0,0,0,0
505411,6,53,1,1,0,6,0,0,0,0,...,0,40,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182635,6,1.3336e+07,9,6,430,11632,430,0,47.7778,143.333,...,1,32,3.00509e+06,0,3.00509e+06,3.00509e+06,5.32584e+06,0,5.32584e+06,5.32584e+06
302265,6,1.00461e+08,7,6,379,11595,373,0,54.1429,140.621,...,2,20,1995,0,1995,1995,1e+08,0,1e+08,1e+08
489358,6,43,1,1,2,6,2,2,2,0,...,0,24,0,0,0,0,0,0,0,0
185546,6,1.08562e+07,9,6,631,11632,631,0,70.1111,210.333,...,1,32,846,0,846,846,5.85365e+06,0,5.85365e+06,5.85365e+06


In [77]:
from catboost import CatBoostClassifier

In [78]:
from lightgbm import LGBMClassifier

In [79]:
from sklearn.ensemble import AdaBoostClassifier

In [80]:
from xgboost import XGBClassifier

In [122]:
clf = CatBoostClassifier(verbose=100, n_estimators=100)

In [175]:
clf = RandomForestClassifier(n_estimators = 1, max_depth = 19, class_weight='balanced')

In [87]:
clf = DecisionTreeClassifier()

In [93]:
y_train_lgm = np.array(y_train_lgm)

In [116]:
clf = AdaBoostClassifier()

In [176]:
clf.fit(train_features, train_label_bi)

RandomForestClassifier(class_weight='balanced', max_depth=19, n_estimators=1)

In [177]:
y_pred = clf.predict(test_features)

In [178]:
df = classification_report(test_label_bi, y_pred, digits=4)
print(df)

              precision    recall  f1-score   support

           0     0.9661    0.9848    0.9753     15000
           1     0.9986    0.9969    0.9978    166967

    accuracy                         0.9959    181967
   macro avg     0.9823    0.9908    0.9866    181967
weighted avg     0.9959    0.9959    0.9959    181967



In [169]:
df = classification_report(test_label, y_pred, digits=4)
print(df)

              precision    recall  f1-score   support

           0     0.9937    0.9636    0.9784     15000
           1     0.9996    0.9995    0.9995     38408
           2     0.9998    0.9995    0.9997     47641
           3     0.8578    0.9761    0.9131       587
           4     0.7500    0.5455    0.6316        11
           5     0.9975    0.9979    0.9977      2380
           6     0.8754    0.9893    0.9289      1769
           7     0.9869    0.9942    0.9905      1739
           8     0.9909    0.9885    0.9897      1650
           9     0.9985    0.9996    0.9990     69037
          10     0.9910    0.9942    0.9926      3088
          11     1.0000    1.0000    1.0000         3
          12     0.6415    0.6571    0.6492       452
          13     0.3490    0.4541    0.3947       196
          14     0.3333    0.1667    0.2222         6

    accuracy                         0.9946    181967
   macro avg     0.8510    0.8484    0.8458    181967
weighted avg     0.9951   

In [114]:
df = classification_report(test_label, y_pred, digits=4)
print(df)

              precision    recall  f1-score   support

           0     0.9965    0.9983    0.9974     15000
           1     0.9999    0.9999    0.9999     38408
           2     0.9998    0.9996    0.9997     47641
           3     0.9879    0.9727    0.9803       587
           4     1.0000    0.4545    0.6250        11
           5     1.0000    1.0000    1.0000      2380
           6     1.0000    0.9960    0.9980      1769
           7     0.9965    0.9942    0.9954      1739
           8     0.9946    0.9964    0.9955      1650
           9     0.9996    0.9998    0.9997     69037
          10     0.9971    0.9955    0.9963      3088
          11     1.0000    1.0000    1.0000         3
          12     0.7646    0.8407    0.8008       452
          13     0.5172    0.3827    0.4399       196
          14     1.0000    0.8333    0.9091         6

    accuracy                         0.9983    181967
   macro avg     0.9502    0.8976    0.9158    181967
weighted avg     0.9982   

In [57]:
roc_auc_score(test_label_bi, y_pred)

NameError: name 'test_label_bi' is not defined

In [66]:
confusion_matrix(test_label, y_pred)

array([[ 3846,  9110,  1281,    18,     0,     6,     4,    23,     1,
          134,   568,     0,     9,     0,     0],
       [    0, 38408,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0],
       [    2,   152, 47487,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0],
       [   20,   555,     0,     0,     0,     0,     0,     0,     0,
           12,     0,     0,     0,     0,     0],
       [    6,     3,     0,     0,     0,     0,     0,     0,     0,
            0,     2,     0,     0,     0,     0],
       [    0,  2379,     1,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0],
       [  901,    10,   852,     0,     0,     0,     0,     0,     0,
            0,     6,     0,     0,     0,     0],
       [   55,  1155,     1,     0,     0,     0,     0,   492,    35,
            0,     1,     0,     0,     0,     0],
       [  141,  1020,   

In [67]:
df = classification_report(test_label, y_pred, digits=4)
print(df)

/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0     0.5461    0.2564    0.3490     15000
           1     0.4834    1.0000    0.6518     38408
           2     0.9169    0.9968    0.9552     47641
           3     0.0000    0.0000    0.0000       587
           4     0.0000    0.0000    0.0000        11
           5     0.0000    0.0000    0.0000      2380
           6     0.0000    0.0000    0.0000      1769
           7     0.2352    0.2829    0.2569      1739
           8     0.4854    0.1715    0.2535      1650
           9     0.9916    0.4883    0.6544     69037
          10     0.3903    0.5696    0.4632      3088
          11     0.0000    0.0000    0.0000         3
          12     0.0000    0.0000    0.0000       452
          13     0.0000    0.0000    0.0000       196
          14     0.0000    0.0000    0.0000         6

    accuracy                         0.6924    181967
   macro avg     0.2699    0.2510    0.2389    181967
weighted avg     0.7766   

/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [132]:
confusion_matrix(test_label7, y_pred)

array([[ 14950,     25,      3,     17,      0,      3,      2],
       [    28, 113891,      4,      0,      0,      0,      2],
       [     2,     13,  47622,      0,      0,      0,      4],
       [    29,      1,      0,    557,      0,      0,      0],
       [     5,      0,      0,      0,      6,      0,      0],
       [    11,      1,      0,      0,      0,   4137,      0],
       [    19,      0,      2,      0,      0,      1,    632]])

In [196]:
df

,precision,recall,f1-score,support
0,0.990939,0.996444,0.993684,9000.000000
1,0.999844,0.999844,0.999844,38408.000000
2,0.999916,0.999664,0.999790,47641.000000
3,0.989437,0.957411,0.973160,587.000000
4,1.000000,0.727273,0.842105,11.000000
5,0.999580,0.999580,0.999580,2380.000000
6,0.998867,0.997174,0.998020,1769.000000
7,0.997692,0.994250,0.995968,1739.000000
8,0.994555,0.996364,0.995459,1650.000000
9,0.999609,0.999739,0.999674,69037.000000


In [127]:
confusion_matrix(test_label7, y_pred)

array([[ 14912,     53,      5,     23,      0,      4,      3],
       [    12, 113906,      2,      0,      0,      0,      5],
       [     4,     13,  47620,      0,      0,      1,      3],
       [    15,      0,      0,    572,      0,      0,      0],
       [     6,      0,      0,      0,      5,      0,      0],
       [     3,      6,      0,      0,      0,   4140,      0],
       [    11,      1,      0,      0,      0,      2,    640]])

In [126]:
df

,precision,recall,f1-score,support
0,0.996592,0.994133,0.995361,15000.000000
1,0.999360,0.999833,0.999596,113925.000000
2,0.999853,0.999559,0.999706,47641.000000
3,0.961345,0.974446,0.967851,587.000000
4,1.000000,0.454545,0.625000,11.000000
5,0.998312,0.997831,0.998071,4149.000000
6,0.983103,0.978593,0.980843,654.000000
accuracy,0.999055,0.999055,0.999055,0.999055
macro avg,0.991223,0.914135,0.938061,181967.000000
weighted avg,0.999056,0.999055,0.999049,181967.000000


In [97]:
df

,precision,recall,f1-score,support
0,0.995188,0.992800,0.993993,15000.000000
1,0.999353,0.999569,0.999461,166967.000000
accuracy,0.999011,0.999011,0.999011,0.999011
macro avg,0.997271,0.996184,0.996727,181967.000000
weighted avg,0.999010,0.999011,0.999010,181967.000000


In [79]:
df = classification_report(test_label, y_pred, digits=4, output_dict=True)
df = pd.DataFrame(df).transpose()

In [81]:
df

,precision,recall,f1-score,support
0,0.994533,0.994533,0.994533,15000.000000
1,0.999844,0.999714,0.999779,38408.000000
2,0.999874,0.999370,0.999622,47641.000000
3,0.962457,0.960818,0.961637,587.000000
4,1.000000,0.454545,0.625000,11.000000
5,0.999159,0.998319,0.998739,2380.000000
6,0.993796,0.996043,0.994918,1769.000000
7,0.994828,0.995400,0.995114,1739.000000
8,0.996962,0.994545,0.995752,1650.000000
9,0.999102,0.999652,0.999377,69037.000000


In [64]:
df

,precision,recall,f1-score,support
0,0.992896,0.997000,0.994944,15000.000000
1,0.999896,0.999662,0.999779,38408.000000
2,0.999811,0.999601,0.999706,47641.000000
3,0.972028,0.947189,0.959448,587.000000
4,1.000000,0.545455,0.705882,11.000000
5,0.999580,0.999160,0.999370,2380.000000
6,0.999431,0.993217,0.996314,1769.000000
7,0.997123,0.996550,0.996836,1739.000000
8,0.996964,0.995152,0.996057,1650.000000
9,0.999392,0.999638,0.999515,69037.000000


In [58]:
train_adj = process(train_src_ip, train_src_port, train_dst_ip, train_dst_port)

82812


In [59]:
train_index = get_torch_data(train_adj)

In [60]:
train_index = torch.FloatTensor(train_index)

In [61]:
len(train_index[0])

45034624

In [62]:
features = train_features.astype(np.float)
features = MinMaxScaler().fit_transform(features)
features = torch.FloatTensor(features)
train_data = Data(edge_index=train_index, features=features, label=train_label)

In [63]:
features

tensor([[3.5294e-01, 8.1262e-03, 2.0101e-04,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.5294e-01, 4.2500e-07, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.5294e-01, 4.2117e-02, 3.0151e-04,  ..., 0.0000e+00, 4.2111e-02,
         4.2111e-02],
        ...,
        [3.5294e-01, 3.6667e-07, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.5294e-01, 9.0468e-02, 8.0402e-04,  ..., 0.0000e+00, 4.8780e-02,
         4.8780e-02],
        [3.5294e-01, 1.6667e-08, 1.0050e-04,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])

In [64]:
train_data = Data(edge_index=train_index, features=features, label=train_label)

In [65]:
train_data

Data(edge_index=[2, 45034624], features=[424589, 78], label=[424589])

In [66]:
train_label = torch.LongTensor(train_label.astype(np.int))

In [67]:
train_data = Data(edge_index=train_index, x=features, y=train_label)

In [68]:
train_data

Data(x=[424589, 78], edge_index=[2, 45034624], y=[424589])

In [69]:
train_data.edge_index = torch.LongTensor(train_data.edge_index.numpy())

In [70]:
train_data.edge_index

tensor([[     0,      0,      0,  ..., 424270, 422950, 423145],
        [401120, 146693, 116550,  ..., 422865, 406454, 284685]])

In [71]:
torch.save(train_data,'cicids2017_inducive_train_data')

In [77]:
test_index = torch.FloatTensor(test_index)

In [78]:
test_label = torch.LongTensor(test_label.astype(np.int))

In [79]:
features = test_features.astype(np.float)
features = MinMaxScaler().fit_transform(features)
features = torch.FloatTensor(features)

In [80]:
test_data = Data(edge_index=test_index, x=features, y=test_label)

In [81]:
test_data.edge_index = torch.LongTensor(test_data.edge_index.numpy())

In [82]:
test_data.edge_index

tensor([[     0,      0,      0,  ..., 179673, 180491, 180869],
        [147456, 163840, 170497,  ...,  54214,  21590,    975]])

In [83]:
torch.save(test_data,'cicids2017_inducive_test_data')

In [73]:
test_adj = process(test_src_ip, test_src_port, test_dst_ip, test_dst_port)

52220


In [74]:
test_index = get_torch_data(test_adj)

In [75]:
test_index = np.array(test_index)

In [76]:
len(test_index[0])

8210378

In [60]:
d = Data(test_index)

In [61]:
d

Data(x=[2, 73762898])

In [45]:
len(test_index[0])

49006

In [50]:
cicids2017_inducive_test_data = torch.load('cicids2017_inducive_test_data')

In [51]:
cicids2017_inducive_test_data

Data(x=[17471, 78], edge_index=[2, 37500536], y=[17471], num_nodes=17471)

In [28]:
train_features.shape

(69882, 78)

In [59]:
test_data = get_lg_graph(test_src_ip, test_src_port, test_dst_ip, test_dst_port, test_features, test_label)

17471


KeyboardInterrupt: 

In [ ]:
train_data = get_lg_graph(train_src_ip, train_src_port, train_dst_ip, train_dst_port, train_features, train_label)

69882


In [56]:
test_data

Data(x=[17471, 78], edge_index=[2, 37500536], y=[17471], num_nodes=17471)

In [57]:
torch.save(test_data,'cicids2017_inducive_test_data')

In [ ]:
torch.save(test_data,'cicids2017_inducive_train_data')

In [107]:
len(df)

149603

In [84]:
src_ip = df[' Source IP']
dst_ip = df[' Destination IP']
src_port = df[' Source Port']
dst_port = df[' Destination Port']

In [85]:
df

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.15-217.68.217.19-54607-443-6,217.68.217.19,443.0,192.168.10.15,54607.0,6.0,03/07/2017 04:35:47,332.0,3.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,192.168.10.1-192.168.10.3-53-60233-17,192.168.10.3,60233.0,192.168.10.1,53.0,17.0,7/7/2017 10:18,96026.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,172.217.12.142-192.168.10.15-80-50621-6,172.217.12.142,80.0,192.168.10.15,50621.0,6.0,5/7/2017 9:54,35.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,172.16.0.1-192.168.10.50-31633-80-6,192.168.10.50,80.0,172.16.0.1,31633.0,6.0,7/7/2017 4:09,74154.0,1.0,5.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,192.168.10.9-119.167.164.11-21068-443-6,119.167.164.11,443.0,192.168.10.9,21068.0,6.0,5/7/2017 2:54,513898.0,4.0,5.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606551,172.16.0.1-192.168.10.50-36182-80-6,172.16.0.1,36182.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,5029253.0,9.0,7.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606552,172.16.0.1-192.168.10.50-36184-80-6,172.16.0.1,36184.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,5013370.0,7.0,6.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606553,172.16.0.1-192.168.10.50-36182-80-6,172.16.0.1,36182.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,18.0,1.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
606554,172.16.0.1-192.168.10.50-36184-80-6,172.16.0.1,36184.0,192.168.10.50,80.0,6.0,6/7/2017 10:35,13.0,1.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13


In [86]:
ori_data

array([['192.168.10.15-217.68.217.19-54607-443-6', '217.68.217.19',
        443.0, ..., 0.0, 0.0, 0],
       ['192.168.10.1-192.168.10.3-53-60233-17', '192.168.10.3', 60233.0,
        ..., 0.0, 0.0, 0],
       ['172.217.12.142-192.168.10.15-80-50621-6', '172.217.12.142',
        80.0, ..., 0.0, 0.0, 0],
       ...,
       ['172.16.0.1-192.168.10.50-36182-80-6', '172.16.0.1', 36182.0,
        ..., 0.0, 0.0, 13],
       ['172.16.0.1-192.168.10.50-36184-80-6', '172.16.0.1', 36184.0,
        ..., 0.0, 0.0, 13],
       ['172.16.0.1-192.168.10.50-36190-80-6', '172.16.0.1', 36190.0,
        ..., 0.0, 0.0, 13]], dtype=object)

In [87]:
new_x = np.delete(ori_data, 6, axis=-1)

In [88]:
feature = new_x[:, 5:-1]

In [89]:
feature.shape

(606556, 78)

In [90]:
label = ori_data[:, -1]

In [91]:
label.shape

(606556,)

In [ ]:
data = get_lg_graph(src_ip, src_port, dst_ip, dst_port, feature, label)

606556


In [56]:
data

Data(x=[189603, 78], edge_index=[2, 107304656], y=[189603], num_nodes=189603)

In [57]:
np.isnan(data.x.numpy()).any()

False

In [58]:
data.train_mask = torch.LongTensor(train_mask)
data.test_mask = torch.LongTensor(test_mask)

In [59]:
torch.save(data, 'cic2017_directed_lg_big_data')

In [ ]:
torch.save(data, 'cic2017_directed_lg_data_15label')

In [18]:
src_ip = np.array(src_ip)
dst_ip = np.array(dst_ip)
src_port = np.array(src_port)
dst_port = np.array(dst_port)

In [19]:
src_nodes = list(zip(src_ip, src_port))
dst_nodes = list(zip(dst_ip, dst_port))

In [20]:
socket_dict = list(zip(src_nodes, dst_nodes))

In [43]:
label = df[' Label']

In [44]:
x = df.drop(columns=[' Label', 'Flow ID',' Source IP',' Source Port', ' Destination IP', ' Destination Port',' Timestamp'])

In [45]:
x = x.replace(['Infinity','infinity ','inf', '-inf','INF','NaN','nan'], 0)

In [46]:
x = (x - x.min())/(x.max()-x.min())

In [47]:
from sklearn.preprocessing import *

In [48]:
x.shape

(117307, 78)

In [49]:
mms = MinMaxScaler()

In [50]:
x_new = mms.fit_transform(x)

/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [51]:
df2 = pd.DataFrame(x_new)
df2 = df2.fillna(0)

In [52]:
features = np.array(df2)

In [80]:
contain_nan = (True in np.isnan(features))
contain_nan

False

In [32]:
all_nodes = set(src_nodes).union(set(dst_nodes))

In [33]:
nodes_to_no_dict = dict()
for index,value in enumerate(all_nodes):
    nodes_to_no_dict[value] = index

In [34]:
num_edge = len(src_nodes)
num_edge

87390

In [35]:
socket_adj_map = {}
socket_no_map = {}
for index, value in enumerate(socket_dict):
    # 获取发出者编号
    s, d = value[0], value[1]
    s_no = nodes_to_no_dict[s]
    d_no = nodes_to_no_dict[d]
    if s_no not in socket_adj_map:
        socket_adj_map[s_no] = []
    if s_no not in socket_no_map:
        socket_no_map[s_no] = []
    socket_adj_map[s_no].append(d_no)
    socket_no_map[s_no].append(index)

In [36]:
adj_list = {}

In [37]:
def build_edge_index(arr):
    for i in range(len(arr)):
        if arr[i] not in adj_list:
            adj_list[arr[i]] = set()
        for j in range(len(arr)):
            if i != j:
                adj_list[arr[i]].add(arr[j])
        

In [38]:
for index, value in socket_no_map.items():
    build_edge_index(value)

In [39]:
for key, value in socket_adj_map.items():
    for i in range(len(value)):
        adj_no = value[i]
        no = socket_no_map[key][i]
        if adj_no in socket_adj_map and len(socket_adj_map[adj_no]) != 0:
            for adj_v in socket_adj_map[adj_no]:
                adj_list[no].add(adj_v)
                adj_list[adj_v].add(no)
        
        


In [40]:
import networkx as nx
G = nx.from_dict_of_lists(adj_list)

In [41]:
import torch
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.utils.convert import to_networkx, from_networkx
from sklearn.model_selection import train_test_split

In [42]:
G

In [43]:
features = torch.tensor(features, dtype=torch.float)

In [44]:
label = torch.tensor(label, dtype=torch.long)

In [52]:
from deepsnap.graph import Graph
from deepsnap.dataset import GraphDataset

In [46]:
H = Graph(G)

D:\anaconda\envs\lightgbm_catboost\lib\site-packages\deepsnap\graph.py:522: UserWarning: Node related key is required.
  warnings.warn("Node related key is required.")


In [47]:
H

Graph(G=[], edge_index=[2, 2918564], edge_label_index=[2, 2918564], node_label_index=[87390])

In [48]:
H.node_feature = features

In [49]:
H

Graph(G=[], edge_index=[2, 2918564], edge_label_index=[2, 2918564], node_feature=[87390, 78], node_label_index=[87390])

In [50]:
H.node_label = label

In [51]:
H

Graph(G=[], edge_index=[2, 2918564], edge_label_index=[2, 2918564], node_feature=[87390, 78], node_label=[87390], node_label_index=[87390])

In [75]:
dataset = GraphDataset([H, H], task='node', minimum_node_per_graph=0)

In [76]:
train, test = dataset.split(transductive=False, split_ratio=[0.75, 0.25])

In [77]:
train[0]

Graph(G=[], edge_index=[2, 2918564], edge_label_index=[2, 2918564], node_feature=[87390, 78], node_label=[87390], node_label_index=[87390], task=[])

In [78]:
test[0]

Graph(G=[], edge_index=[2, 2918564], edge_label_index=[2, 2918564], node_feature=[87390, 78], node_label=[87390], node_label_index=[87390], task=[])

In [41]:
data = from_networkx(G)

In [42]:
data

Data(edge_index=[2, 2918279], num_nodes=87390)

In [59]:
data

Data(x=[117307, 78], edge_index=[2, 32884128], y=[117307], num_nodes=117307)

In [60]:
alltrainidx = np.arange(data.num_nodes)

In [61]:
alltrainidx.shape

(117307,)

In [44]:
data.x = features
data.y = label

In [62]:
train_mask, test_mask = train_test_split(alltrainidx, test_size=0.25, stratify=label, random_state=1)

In [63]:
train_mask = torch.tensor(train_mask, dtype=torch.long)
test_mask = torch.tensor(test_mask, dtype=torch.long)

In [64]:
data.train_mask = train_mask
data.test_mask = test_mask

In [89]:
torch.save(data, 'CIC2017_data')

In [48]:
data.has_isolated_nodes()

True

In [49]:
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv,ResGatedGraphConv,GATv2Conv,SAGEConv,GCNConv
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.loader import NeighborSampler, NeighborLoader
import copy
import tqdm

In [66]:
num_class = 12
features_num = 78
hidden1 = 32
hidden2 = 16

In [65]:
device = torch.device('cuda:0')
data = data.to(device)

In [82]:
train_loader = NeighborLoader(data, input_nodes = data.train_mask, num_neighbors=[10]*10, batch_size=1024, shuffle=True)

In [80]:
batch0 = next(iter(train_loader))

In [81]:
batch0

Data(edge_index=[2, 88401], num_nodes=12019, x=[12019, 78], y=[12019], train_mask=[65542], test_mask=[21848], batch_size=1024)

In [83]:
subgraph_loader = NeighborLoader(copy.copy(data), input_nodes=None,num_neighbors=[-1], batch_size=1024, shuffle=False)
del subgraph_loader.data.x, subgraph_loader.data.y
# Add global node index information.
subgraph_loader.data.num_nodes = data.num_nodes
subgraph_loader.data.n_id = torch.arange(data.num_nodes)

In [84]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, in_channels))
        self.convs.append(GATConv(in_channels, in_channels))
        self.convs.append(GATConv(in_channels, in_channels))
        self.convs.append(GATConv(in_channels, in_channels))
        self.convs.append(GATConv(in_channels, in_channels))
        self.convs.append(GATConv(in_channels, in_channels))
        self.convs.append(GATConv(in_channels, in_channels))
        self.convs.append(GATConv(in_channels, in_channels))
        self.convs.append(GATConv(in_channels, in_channels))
        self.convs.append(GATConv(in_channels, out_channels))

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < len(self.convs) - 1:
                x = x.relu_()
                x = F.dropout(x, p=0.1, training=self.training)
        return x

    @torch.no_grad()
    def inference(self, x_all, subgraph_loader):
        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch:
        for i, conv in enumerate(self.convs):
            xs = []
            for batch in subgraph_loader:
                x = x_all[batch.n_id.to(x_all.device)].to(device)
                x = conv(x, batch.edge_index.to(device))
                if i < len(self.convs) - 1:
                    x = x.relu_()
                xs.append(x[:batch.batch_size].cpu())
            x_all = torch.cat(xs, dim=0)
        return x_all


In [85]:
model = Net(features_num, hidden2, num_class).to(device)  # 定义GAT
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [86]:
def train():
    total_loss = total_correct = total_examples = 0
    for batch in train_loader:
        optimizer.zero_grad()
        y = batch.y[:batch.batch_size]
        y_hat = model(batch.x, batch.edge_index.to(device))[:batch.batch_size]
        loss = F.cross_entropy(y_hat, y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * batch.batch_size
        total_correct += int((y_hat.argmax(dim=-1) == y).sum())
        total_examples += batch.batch_size
    print(total_loss / total_examples)

    return total_loss / total_examples, total_correct / total_examples

In [87]:
from sklearn.metrics import f1_score, classification_report

In [88]:
model.train()
loss_all = []
epochs = 100
for i in range(epochs):
    loss, acc = train()
    loss_all.append(loss)

1.003401975942678
0.3490186337216657
0.24521065540030856
0.22718077614380083
0.18771160698356068
0.17604750563033864
0.15304803150475202
0.15089697698653262
0.18347311543263747
0.1632280812799489


KeyboardInterrupt: 

In [77]:
model.eval()
y_hat = model.inference(data.x, subgraph_loader).argmax(dim=-1)
y = data.y.to(y_hat.device)
cr1 = classification_report(y[test_mask], y_hat[test_mask],digits=4)
cr2 = classification_report(y[train_mask], y_hat[train_mask],digits=4)
print(cr1)
print(cr2)

              precision    recall  f1-score   support

           0     0.9750    0.9842    0.9796      4998
           1     0.9948    0.9940    0.9944      2500
           2     0.9984    0.9972    0.9978      2500
           3     0.9731    0.9573    0.9652       492
           4     0.8889    0.8889    0.8889         9
           5     0.9964    0.9894    0.9929      1985
           6     0.9966    0.9824    0.9894      1474
           7     0.9958    0.9917    0.9938      1449
           8     0.9913    0.9905    0.9909      1375
           9     0.9872    0.9904    0.9888      2490
          10     0.9953    0.9953    0.9953      2573
          11     1.0000    1.0000    1.0000         3

    accuracy                         0.9894     21848
   macro avg     0.9827    0.9801    0.9814     21848
weighted avg     0.9895    0.9894    0.9894     21848

              precision    recall  f1-score   support

           0     0.9756    0.9906    0.9830     14995
           1     0.9973 